# АНСАМБЛИ

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/pytyon_exam_preprosessed.csv')
df.head()

,student_id,age,country,prior_programming_experience,weeks_in_course,hours_spent_learning_per_week,practice_problems_solved,projects_completed,tutorial_videos_watched,uses_kaggle,participates_in_discussion_forums,debugging_sessions_per_week,self_reported_confidence_python,passed_exam
0,1,54,Pakistan,Beginner,12,12.2,60,1,24,0,1,4,10,0
1,2,44,Nigeria,NaN,15,14.2,52,1,32,0,0,5,7,0
2,3,30,India,Intermediate,5,7.7,71,1,43,0,0,6,3,0
3,4,23,Nigeria,Beginner,4,5.6,64,2,40,1,0,3,10,0
4,5,36,UK,NaN,3,3.5,62,1,47,1,0,3,7,0


In [ ]:
Y = df['passed_exam']
X = df.drop(['passed_exam'], axis=1)

In [ ]:
# закодируем категориальные и сделаем скейлер
categorical = X.describe(include='object').columns
numerical = X.describe().columns

In [ ]:
from sklearn.preprocessing import TargetEncoder, StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer([
    ('encoder', TargetEncoder(), categorical),
    ('scaler', MinMaxScaler(), numerical)], remainder='passthrough')
transformer.set_output(transform="pandas")
X_scaled = transformer.fit_transform(X, Y)

In [ ]:
X_scaled.describe()

,encoder__country,encoder__prior_programming_experience,scaler__student_id,scaler__age,scaler__weeks_in_course,scaler__hours_spent_learning_per_week,scaler__practice_problems_solved,scaler__projects_completed,scaler__tutorial_videos_watched,scaler__uses_kaggle,scaler__participates_in_discussion_forums,scaler__debugging_sessions_per_week,scaler__self_reported_confidence_python
count,3000.000000,3000.000000,3000.00000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,0.177554,0.176944,0.50000,0.508605,0.508643,0.411419,0.480455,0.222370,0.474924,0.400667,0.498333,0.292549,0.507333
std,0.032254,0.189414,0.28882,0.297426,0.305358,0.171704,0.148328,0.155078,0.142347,0.490115,0.500081,0.131572,0.320579
min,0.099371,0.002396,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.155024,0.008169,0.25000,0.263158,0.285714,0.292398,0.384615,0.111111,0.363636,0.000000,0.000000,0.176471,0.222222
50%,0.176215,0.125645,0.50000,0.526316,0.500000,0.409357,0.480769,0.222222,0.477273,0.000000,0.000000,0.294118,0.555556
75%,0.204791,0.349768,0.75000,0.763158,0.785714,0.526316,0.576923,0.333333,0.568182,1.000000,1.000000,0.352941,0.777778
max,0.235244,0.631886,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, train_size = 0.8)

Реализуем баггинг
(Сначала единственное дерево для сравнения)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn import ensemble
import warnings
import plotly.express as px
from sklearn.metrics import confusion_matrix, classification_report
warnings.filterwarnings('ignore')

In [ ]:
single_tree = tree.DecisionTreeClassifier(max_depth=8, random_state=1).fit(X_train, Y_train)
y_pred = single_tree.predict(X_test)

In [ ]:
plt.rcParams['figure.figsize'] = (10,10)
fig = px.imshow(confusion_matrix(Y_test, y_pred), text_auto=True)
fig.update_layout(xaxis_title='Target', yaxis_title='Prefiction')

In [ ]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.90      0.90       492
           1       0.56      0.58      0.57       108

    accuracy                           0.84       600
   macro avg       0.74      0.74      0.74       600
weighted avg       0.85      0.84      0.84       600



Строится n независимых друг от друга
моделей, затем, в случае решения задачи регрессии, их результаты
усредняются, в случае классификации – берется наиболее встречаемый
результат моделей.


In [ ]:
len(X_train)

2400

In [ ]:
n = 10
X_train_sample = np.array_split(X_train, n)
Y_train_sample = np.array_split(Y_train, n)

In [ ]:
models = []
for i in range(n):
  model_tree = tree.DecisionTreeClassifier(max_depth=8, random_state=1)
  model_tree.fit(X_train_sample[i], Y_train_sample[i])
  models.append(model_tree)

In [ ]:
models

[DecisionTreeClassifier(max_depth=8, random_state=1),
 DecisionTreeClassifier(max_depth=8, random_state=1),
 DecisionTreeClassifier(max_depth=8, random_state=1),
 DecisionTreeClassifier(max_depth=8, random_state=1),
 DecisionTreeClassifier(max_depth=8, random_state=1),
 DecisionTreeClassifier(max_depth=8, random_state=1),
 DecisionTreeClassifier(max_depth=8, random_state=1),
 DecisionTreeClassifier(max_depth=8, random_state=1),
 DecisionTreeClassifier(max_depth=8, random_state=1),
 DecisionTreeClassifier(max_depth=8, random_state=1)]

In [ ]:
y_pred = []
for i in range(len(models)):
  y_pred.append(models[i].predict(X_test))

In [ ]:
# нам нужен наиболее встречающийся результат тк классификация
from scipy import stats

mode_pred = stats.mode(np.array(y_pred))
mode_pred[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [ ]:
plt.rcParams['figure.figsize'] = (10,10)
fig = px.imshow(confusion_matrix(Y_test, mode_pred[0]), text_auto=True)
fig.update_layout(xaxis_title='Target', yaxis_title='Prefiction')

In [ ]:
print(classification_report(Y_test, mode_pred[0]))

              precision    recall  f1-score   support

           0       0.88      0.97      0.92       492
           1       0.74      0.40      0.52       108

    accuracy                           0.87       600
   macro avg       0.81      0.68      0.72       600
weighted avg       0.86      0.87      0.85       600



Используем модель RandomForestClassifier


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()
param_grid = {
    'max_depth': [12, 18],
    'min_samples_leaf': [3, 10],
    'min_samples_split': [6, 12]
}
Grid_model = GridSearchCV(estimator=random_forest, param_grid=param_grid,
                          scoring='f1_macro', cv=4)
Grid_model.fit(X_train, Y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [12, 18], 'min_samples_leaf': [3, 10],
                         'min_samples_split': [6, 12]},
             scoring='f1_macro')

In [ ]:
best_model = Grid_model.best_estimator_
best_model

RandomForestClassifier(max_depth=12, min_samples_leaf=3, min_samples_split=6)

In [ ]:
y_pred = best_model.predict(X_test)
plt.rcParams['figure.figsize'] = (10,10)
fig = px.imshow(confusion_matrix(Y_test, y_pred), text_auto=True)
fig.update_layout(xaxis_title='Target', yaxis_title='Prefiction')

In [ ]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.96      0.92       492
           1       0.72      0.45      0.56       108

    accuracy                           0.87       600
   macro avg       0.80      0.71      0.74       600
weighted avg       0.86      0.87      0.86       600



Реализируем бустинг c CatBoostClassifier на тех же данных, что использовались для баггинга.

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.4 MB/s eta 0:00:00


In [ ]:
import catboost as cb
model_catboost = cb.CatBoostClassifier(iterations=3000, task_type='GPU', devices='0')
model_catboost.fit(X_train, Y_train)

Learning rate set to 0.012005
0:	learn: 0.6789551	total: 82.2ms	remaining: 4m 6s
1:	learn: 0.6661371	total: 150ms	remaining: 3m 44s
2:	learn: 0.6501759	total: 231ms	remaining: 3m 50s
3:	learn: 0.6374832	total: 308ms	remaining: 3m 51s
4:	learn: 0.6225021	total: 368ms	remaining: 3m 40s
5:	learn: 0.6088788	total: 445ms	remaining: 3m 42s
6:	learn: 0.5944260	total: 503ms	remaining: 3m 35s
7:	learn: 0.5833219	total: 580ms	remaining: 3m 37s
8:	learn: 0.5719401	total: 651ms	remaining: 3m 36s
9:	learn: 0.5596704	total: 722ms	remaining: 3m 35s
10:	learn: 0.5517166	total: 795ms	remaining: 3m 35s
11:	learn: 0.5397469	total: 869ms	remaining: 3m 36s
12:	learn: 0.5296068	total: 928ms	remaining: 3m 33s
13:	learn: 0.5207427	total: 998ms	remaining: 3m 32s
14:	learn: 0.5152218	total: 1.04s	remaining: 3m 27s
15:	learn: 0.5078256	total: 1.11s	remaining: 3m 27s
16:	learn: 0.4994413	total: 1.18s	remaining: 3m 27s
17:	learn: 0.4937463	total: 1.25s	remaining: 3m 27s
18:	learn: 0.4854619	total: 1.32s	remaining:

In [ ]:
y_pred = model_catboost.predict(X_test, task_type='CPU')

In [ ]:
plt.rcParams['figure.figsize'] = (10,10)
fig = px.imshow(confusion_matrix(Y_test, y_pred), text_auto=True)
fig.update_layout(xaxis_title='Target', yaxis_title='Prefiction')

In [ ]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93       492
           1       0.71      0.65      0.68       108

    accuracy                           0.89       600
   macro avg       0.82      0.79      0.80       600
weighted avg       0.89      0.89      0.89       600



Из всех алгоритмов CatBoostClassifier показал лучший результат